In [2]:
import pandas as pd
import numpy as np

import pyspark
from pyspark.sql.functions import col
from pyspark.sql import SQLContext

import spacy
import string

import re
import gensim
from gensim import corpora

import en_core_web_sm
nlp = en_core_web_sm.load()

In [3]:
spark = (pyspark.sql.SparkSession.builder
    .master("local")
    .getOrCreate())

In [4]:
books_df = spark.read.json('data/goodreads_books_comics_graphic.json')

In [9]:
books_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- role: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- edition_information: string (nullable = true)
 |-- format: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- is_ebook: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- kindle_asin: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- link: string (nullable = true)
 |-- num_pages: string (nullable = true)
 |-- popular_shelves: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- count: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- pub

In [27]:
books = books_df.select(['book_id', 'popular_shelves', 'title'])

In [31]:
shelves = books.select('popular_shelves')

Column<b'popular_shelves[0]'>

In [ ]:
# Functions to clean and tokenize text
def replace_punct_and_numbers(text):
    """Remove punctuation from document"""
    punct = [punc for punc in string.punctuation]
    num = list(range(10))
    clean_text = "".join([letter for letter in text if (letter not in punct) and (letter not in num)])
    return clean_text

def clean(doc):
    """Process text and return tokenized"""
    punct_free = replace_punct_and_numbers(doc)
    doc_ready = nlp(punct_free)
    tokens = [token.lemma_.lower() for token in doc_ready 
                  if (not token.is_stop) and (token.text.strip() != "")]
    return tokens